# SEN163A - Fundamentals of Data Analytics
# Assignment 2 - Large-scale Internet Data Analysis
### Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
### Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl)

### 05-03-2022
## Group 2
- Emmanuel M Boateng - '5617642'
- Joost Oortwijn - '4593472'
- Philip Busscher - ''4611993''
- Floris Kool - ''4975243''


# Introduction
short description of the assignment

# 1. Dataset description

Short description of the 4 datasets used

## 1.1 Opening the data

In [2]:
import pickle
import time
import bz2
import os
import sys
import json
import pandas

In [3]:
#AS Dataset

AS_Filename = 'data/AS_dataset.pkl'

with open(AS_Filename, 'rb') as file:
    
    AS_df = pickle.load(file)

In [4]:
#Probe dataset

Probe_Filename = 'data/probe_dataset.pkl'

with open(Probe_Filename, 'rb') as file:
    
    P_df = pickle.load(file)

In [5]:
#Ripe dataset 

#Method needs to be decided

In [6]:
#IP 2 Location dataset

IP_Filename = "data/IP2LOCATION-LITE-DB1.CSV"

ipv4_df = pandas.read_csv(IP_Filename)

More detailed description of data if needed (Can also be after opening each dataset)

## 1.2 Limitations in data

Evaluate if there are limitations in the provided datasets (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.

...

In [7]:
#Code needed to prove limitations

Some list of limitations in text

# 2 Analysis

Short description of what is going to be analyzed

## 2.1 AS

With the AS and probe data set, find the number m of AS’s that can be used for hosting in the EU
and have probes in the RIPE data set. Sort the ASN’s in ascending order and include the first and last
three in your report (number, name and country).


In [32]:
#Upper part should be removed because run in part 1
import pickle

with open('data/AS_dataset.pkl', 'rb') as file:
    AS_df = pickle.load(file)
    
with open('data/probe_dataset.pkl', 'rb') as file:    
    P_df = pickle.load(file)
    
decomFilename = 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'
decomFile     = open(decomFilename, 'rt')   


RIPEProbes = []
index = 0

#Create list of all probes that are in the RIPE dataset
for line in decomFile:
    jsonline = json.loads(line)
    
    if jsonline["prb_id"] not in RIPEProbes:
        RIPEProbes.append(jsonline["prb_id"])
                          
    #Read only first 1m lines
    index = index + 1
    if index > 1000000:
        break
        
                  
print("Probes in first 1m lines of RIPE Dataset: " +str(len(RIPEProbes)))            

decomFile.close()


Probes in first 1m lines of RIPE Dataset: 11639


In [38]:
index = 0
ProbeASNs = []

#Create list of all probes in both RIPE and probe datasets
#Saves only the ASNs as these are used later
#Probe IDs no longer used after this point
for probe in P_df["prb_id"]:
    if probe in RIPEProbes:
        ProbeASNs.append(P_df["ASN"][index])
        
    index = index + 1
    
print("Probes in both RIPE and probe dataset: " + str(len(ProbeASNs)))    

Probes in both RIPE and probe dataset: 7221


In [39]:
HostProbeASNs = []

index = 0

for ASN in AS_df["ASN"]:
    
    if ASN in ProbeASNs:
        if AS_df["type"][index] == "hosting":
            HostProbeASNs.append(ASN)            
    index = index + 1    
    
print("Amount of probes with an ASN with type hosting: " + str(len(HostProbeASNs))) 

HostProbeASNs.sort()


Amount of probes with an ASN with type hosting: 194


Description of results

## 2.2 Hosting location
For a single hour in the RIPE data set: find all valid entries where the probe has hosting type AS and
the target IPv4 is from an EU country. Implement this in an efficient way.

In [40]:
HostProbes = []

for ProbeASN in HostProbeASNs:
    
    index = 0
    for ASN in P_df["ASN"]:
        if ASN == ProbeASN:
            HostProbes.append(P_df["prb_id"][index])
            break
        index = index + 1
        
print(len(HostProbes))

194


Some probes didn't have a destination addr in the RIPE dataset

In [41]:
decomFilename = 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'
decomFile     = open(decomFilename, 'rt')   


HostIPs = []
index = 0

for line in decomFile:
    jsonline = json.loads(line)
    
    
    if jsonline["prb_id"] in HostProbes:
        try:
            #Check for duplicates
            if jsonline["dst_addr"] not in HostIPs:
                #Check if IP is of type 4
                if jsonline["af"] == 4:
                    HostIPs.append(jsonline["dst_addr"])
        except KeyError as err:
            pass
    
    #Read only first 1m lines
    index = index + 1
    if index > 1000000:
        break
                              
print("Amount of IPs in the RIPE data connected to an AS of type Hosting: " + str(len(HostIPs)))

decomFile.close()

Amount of IPs in the RIPE data connected to an AS of type Hosting: 1047


In [46]:
#Converting the xxx.xxx.xxx.xxx format of the host IPs to integer
#Needed for when comparing to the IPv4 dataset

HostIPs_Integer = []


for IPString in HostIPs:
    IP_Splitstring = IPString.split(".") 
 
    HostIPs_Integer.append(int(IP_Splitstring[0]) * 16581375 + int(IP_Splitstring[1]) * 65025 + int(IP_Splitstring[2]) * 255 + int(IP_Splitstring[3]))  

print(len(HostIPs_Integer))



1047


In [47]:
#List of country codes that are an EU member

EU_Countries = ["AT",
    "BE",
    "HR",
    "CY",
    "CZ",
    "DK",
    "EE",
    "FI",
    "FR",
    "GR",
    "DE",
    "HU",
    "IE",
    "IT",
    "LV",
    "LT",
    "LU",
    "MT",
    "NL",
    "PL",
    "PT",
    "RO",
    "SK",
    "SI",
    "ES",
    "SE"]

Please check below

In [48]:
#Compare IPv4 with HostIPs_Integer

ipv4_df = pandas.read_csv("data/IP2LOCATION-LITE-DB1.CSV")
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True)

ipv4_df.head()

HostIPs_EU = []
index = 0
ipv4index = 0

#Sorting the IP list so we can check from low to high IPs
HostIPs_Integer.sort

for IP_to in ipv4_df["ip_to"]:
    
    while HostIPs_Integer[index] < IP_to:
        if ipv4_df["country_code"][ipv4index] in EU_Countries:
            HostIPs_EU.append(HostIPs_Integer[index])
            print(HostIPs_Integer[index])
            print(ipv4_df["country_code"][ipv4index])
        index = index + 1
        if index >= len(HostIPs_Integer):
            break;
    ipv4index = ipv4index + 1  
    if index >= len(HostIPs_Integer):
        break;
                  

print(len(HostIPs_EU))


88072507
HU
1


In [112]:
ipv4_df.head()

,ip_from,ip_to,country_code,country_name
0,16777216,16777471,US,United States of America
1,16777472,16778239,CN,China
2,16778240,16779263,AU,Australia
3,16779264,16781311,CN,China
4,16781312,16785407,JP,Japan


Description of results

## 2.3 Latency
Move from using only an hour to the full day. It is advisable to store the raw results of each file. Then,
using all processed files, calculate the average latency’s for each country-AS combination and store
the results into one ncountries ×m matrix. If we could place one server in each country, what would the
minimum average latency be for each country? (include in your report)


In [ ]:
#Code...

Description of results

## 2.4 Optimal server locations
Since we are only allowed to place four servers, determine the best four datacenters based on the total
latency for all countries. Report your findings and your procedure to obtain them. Also include the
average latency for each country.


In [ ]:
#Code...

0Description of results

# Conclusions

... 
add code if needed

In [ ]:
index = 0
ProbeASNs = []

#Create list of all probes in both RIPE and probe datasets
#Saves only the ASNs as these are used later
#Probe IDs no longer used after this point
for probe in P_df["prb_id"]:
    if probe in RIPEProbes:
        ProbeASNs.append(P_df["ASN"][index])
        
    index = index + 1
    
print("Probes in both RIPE and probe dataset: " + str(len(ProbeASNs)))    

In [ ]:
HostProbes = []

index = 0

print(len(HostProbeASNs))

for ASN in P_df["ASN"]:
    if ASN in HostProbeASNs:
        if P_df["prb_id"][index] not in HostProbes:
            HostProbes.append(P_df["prb_id"][index])
    index = index + 1

print(len(HostProbes))    

In [ ]:
0#Random stuff I didn't want to throw away yet
#0Code for finding all host probes from EU in the dataset of one hour
import time
import bz2
import os
import sys
import json

# open decompressed file
decomFilename = 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'
decomFile     = open(decomFilename, 'rt') 

#read first line and print
#firstLine = decomFile.readline();
#print(firstLine)

#the line appears to be json-formatted: pretty print json
#firstLineJson = json.loads(firstLine)

#read all lines of first file
count = 0
st    = time.time()
for line in decomFile:
    jsonline = json.loads(line)
    #print(json.dumps(jsonline, sort_keys=True, indent=4))
    count = count + 1
    if count > 10000: 
          break

#print the last line
print(json.dumps(jsonline, sort_keys=True, indent=4))

#print the read duration
dur         = round(time.time() - st,2)
print("Loading took: " + str(dur) + " seconds")
print("The file had " + str(count) + "lines")

#finally close decomFile
decomFile.close()